In [ ]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit import RDConfig
from rdkit import DataStructs
from rdkit.Chem.Fingerprints import FingerprintMols
from itertools import *

d1 = pd.read_csv('/home/uxue/Documents/Master_Thesis/Second_step/scaffold_extraction/input_files/Homo sapiens.txt', sep="\t", header=0)
d2 = pd.read_csv('/home/uxue/Documents/Master_Thesis/Second_step/scaffold_extraction/input_files/Mus musculus.txt', sep="\t", header=0)
d3 = pd.read_csv('/home/uxue/Documents/Master_Thesis/Second_step/scaffold_extraction/input_files/Bos taurus.txt', sep="\t", header=0)
d4 = pd.read_csv('/home/uxue/Documents/Master_Thesis/Second_step/scaffold_extraction/input_files/Rattus norvegicus.txt', sep="\t", header=0)

df = pd.concat([d1,d2,d3,d4])
#d1



#df

#Save into same .txt file

#df.to_csv('/home/uxue/Documents/Master_Thesis/Second_step/scaffold_extraction/input_files/tab.txt', header=None, index=True, sep='\t')


In [ ]:
#Resetear los indices para que se numeren correctamente
df = df.reset_index(drop=True)
#df

In [ ]:
#Find substructures and group them in a dictionary

lista=set(df.smiles)
suppl=[Chem.MolFromSmiles(s) for s in lista]
match_dictionary={}
for m, n in permutations(suppl, 2):
    mm=Chem.MolToSmiles(m)
    nn=Chem.MolToSmiles(n)
    if nn not in match_dictionary.keys():
        match_dictionary[nn]=[]
    if m.HasSubstructMatch(n):
        match_dictionary[nn].append(mm)

#Remove keys with empty lists
match_dictionary={k:v for k,v in match_dictionary.items() if len(v)>0}


#Remove subgroups an leave only the big group with the biggest smile as key
remove=[]
base_keys = match_dictionary.keys()
for key, values in match_dictionary.items():
    for words in values:
        for base_key in base_keys:
            if base_key in words:
                remove.append(base_key)

for i in remove:
    if i in match_dictionary.keys():
        del match_dictionary[i]




In [ ]:
vals=match_dictionary.values()
print(list(vals))

In [ ]:
#Sort dict by len(values)
sorted(numbers.values())

In [ ]:
#Visualize results
from rdkit import Chem
from rdkit.Chem import PandasTools
import pandas as pd
import os
from rdkit import RDConfig

sm=(list(match_dictionary.keys()))


df_n = pd.DataFrame({'Smiles': sm})
PandasTools.AddMoleculeColumnToFrame(df_n,'Smiles','Molecule',includeFingerprints=True)

df_n

In [ ]:
#Add a column for the scaffolds
df['scaffold']=None
scaffolds=[]
for i, row in df.iterrows():
    sc=[]
    for key, values in match_dictionary.items():
        if row['smiles'] in values:
            sc.append(key)
    if len(sc)==0:
        sc.append(row['smiles'])
    
    scaffolds.append(sc)

df.scaffold=scaffolds

df


In [ ]:
#Create new df with the smiles and corresponding scaffold for each one
df1=pd.DataFrame({},columns=['freq','normfreq','organism','smiles','scaffold'])

for i, row in df.iterrows():
    sc=[]
    for key, values in match_dictionary.items():
        if row['smiles'] in values:
            sc.append(key)
    if len(sc)==0:
        sc.append(row['smiles'])
    for s in sc:
        df1=df1.append({'freq':row['freq'],'normfreq':row['normfreq'], 'organism':row['organism'], 'smiles':row['smiles'],'scaffold': s},ignore_index=True)

df1

In [ ]:
sm=(list(df1.scaffold))


df_n = pd.DataFrame({'Smiles': sm})
PandasTools.AddMoleculeColumnToFrame(df_n,'Smiles','Molecule',includeFingerprints=True)

df_n

In [ ]:
#Create a graph to visulaize the scaffolds but not separation by species
df1.groupby('scaffold').size().reset_index().sort_values(0,ascending=False).head(50).plot.bar(figsize=(110,60))

# Add title and axis names
plt.title('AChE')
plt.ylabel('%')

plt.rc('font', size=50)          # controls default text sizes
plt.rc('axes', titlesize=70)     # fontsize of the axes title
plt.rc('axes', labelsize=60)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=70)    # fontsize of the tick labels
plt.rc('ytick', labelsize=70)    # fontsize of the tick labels
plt.rc('legend', fontsize=70)    # legend fontsize
plt.rc('figure', titlesize=100)  # fontsize of the figure title
plt.show()
plt.close()

In [ ]:
#Save data in .txt table and plot 
df4 = pd.DataFrame(index=df1["scaffold"].unique(), columns=df1["organism"].unique())

for i, row in df1.iterrows():
    df4.loc[row["scaffold"], row["organism"]] = row["normfreq"]

df4.to_csv('/home/uxue/Documents/Master_Thesis/Definitivos/2/results2.txt', header=1, index=True, sep='\t')
#df4.plot.bar(figsize=(110,60))
#
## Add title and axis names
#plt.title('DRD2')
#plt.ylabel('%')
#
#plt.rc('font', size=50)          # controls default text sizes
#plt.rc('axes', titlesize=70)     # fontsize of the axes title
#plt.rc('axes', labelsize=60)    # fontsize of the x and y labels
#plt.rc('xtick', labelsize=70)    # fontsize of the tick labels
#plt.rc('ytick', labelsize=70)    # fontsize of the tick labels
#plt.rc('legend', fontsize=70)    # legend fontsize
#plt.rc('figure', titlesize=100)  # fontsize of the figure title
#
#plt.savefig('/home/uxue/Documents/Master_Thesis/fig2.png')
#plt.show()
#plt.close()
df4


In [ ]:
##### Save data in .txt table and plot 
df5 = pd.DataFrame(index=df1["scaffold"].unique(), columns=df1["organism"].unique())

for i, row in df1.iterrows():
    df5.loc[row["smiles"], row["organism"]] = row["scaffold"]

df5.to_csv('/home/uxue/Documents/Master_Thesis/Definitivos/2/results1.txt', header=1, index=True, sep='\t')
df5

In [ ]:
#description=
df5.describe()
#description.to_csv('/home/uxue/Documents/Master_Thesis/Definitivos/2/description1.txt', header=1, index=True, sep='\t')

In [ ]:
##### Save data in .txt table and plot 
df6 = pd.DataFrame(index=df1["scaffold"].unique(), columns=df1["organism"].unique())

for i, row in df1.iterrows():
    df6.loc[row["smiles"], row["organism"]] = row["scaffold"]

df6.to_csv('/home/uxue/Documents/Master_Thesis/Definitivos/2/results2.txt', header=1, index=True, sep='\t')
df6

In [ ]:
description=df6.describe()
description.to_csv('/home/uxue/Documents/Master_Thesis/Definitivos/2/description2.txt', header=1, index=True, sep='\t')